In [5]:
#importing Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time


import torch
from torchtext.data import Field, TabularDataset, BucketIterator, Iterator
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader

import warnings
warnings.filterwarnings('ignore')

In [6]:
device = torch.device('cpu')
tokenizer = RobertaTokenizer.from_pretrained(r"\\rsc.humad.com\rdrive\FinRec\FRSolutions\FRS Reporting\RXS Information\Financial Reporting\Forecast Research Dashboard\Data Science\Member_grievance\Abhirup\Model\roberta-base",return_dict=False)
output_path=r"C:\Users\KPA2877\Documents\Multiclass"
data_path=r"C:\Users\KPA2877\Documents\Multiclass"

In [7]:
df=pd.read_excel(r"C:\Users\KPA2877\Documents\Multiclass\Multiclass1.xlsx")
df.head()

Sentence            label
0  — diego the called because she is not ok with ...  Benefit Package
1  ci for assistance he does nt agree with a copa...  Benefit Package
2  tamara ticket ticket 2122024 16808 is really u...  Benefit Package
3  added email address reviewed claim port resp c...  Benefit Package
4  apologized to and filed greivance and agent ar...  Benefit Package

In [8]:
df.to_csv(r"C:\Users\KPA2877\Documents\Multiclass\Multiclass.csv")

In [9]:
# Model with extra layers on top of RoBERTa
class ROBERTAClassifier(torch.nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(ROBERTAClassifier, self).__init__()
        
        self.roberta = RobertaModel.from_pretrained(r"\\rsc.humad.com\rdrive\FinRec\FRSolutions\FRS Reporting\RXS Information\Financial Reporting\Forecast Research Dashboard\Data Science\Member_grievance\Abhirup\Model\roberta-base")
        self.d1 = torch.nn.Dropout(dropout_rate)
        self.l1 = torch.nn.Linear(768, 64)
        self.bn1 = torch.nn.LayerNorm(64)
        self.d2 = torch.nn.Dropout(dropout_rate)
        self.l2 = torch.nn.Linear(64, 5)
        
    def forward(self, input_ids, attention_mask):
        _, x = self.roberta(input_ids=input_ids, attention_mask=attention_mask,return_dict=False)
        x = self.d1(x)
        x = self.l1(x)
        x = self.bn1(x)
        x = torch.nn.Tanh()(x)
        x = self.d2(x)
        x = self.l2(x)
        
        return x  

In [10]:
# Set tokenizer hyperparameters.
MAX_SEQ_LEN = 128
BATCH_SIZE = 16
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)
# Define columns to read.
label_field = Field(sequential=False, use_vocab=False, batch_first=True)
text_field = Field(use_vocab=False, 
                   tokenize=tokenizer.encode, 
                   include_lengths=False, 
                   batch_first=True,
                   fix_length=MAX_SEQ_LEN, 
                   pad_token=PAD_INDEX, 
                   unk_token=UNK_INDEX)

fields = {'Sentence' : ('Sentence', text_field), 'label' : ('label', label_field)}
#print(train_data)

In [11]:
import random
minor_classes = [0,1, 3, 4] 

# Define a custom insertion function for data augmentation
# def insert_word(tokens, word):
#     idx = random.randint(0, len(tokens))  # Choose a random index to insert the word
#     tokens.insert(idx, word)  # Insert the word at the chosen index
#     return tokens  # Return the augmented list of tokens

# # Apply data augmentation only to examples belonging to minor classes
# for example in train_data.examples:
#     if example.label in minor_classes:
#         tokens = example.Sentence  # Retrieve the list of tokens
#         augmented_tokens = insert_word(tokens, 'augmentation')  # Insert a word
#         example.Sentence = augmented_tokens 

In [12]:
def augment_data(sentence):
    # Implement your data augmentation logic here
    # For example, inserting a word
    words = sentence.split()
    idx = random.randint(0, len(words))  # Choose a random index to insert the word
    words.insert(idx, 'augmentation')  # Insert the word at the chosen index
    augmented_sentence = ' '.join(words)
    return augmented_sentence

In [13]:
# Apply data augmentation for minor classes
minor_classes = [0,1,3,4]
#df['augmented_sentence'] = df.apply(lambda row: augment_data(row['Sentence']) if row['label'] in minor_classes else row['Sentence'], axis=1)
df['augmented_sentence'] = df.apply(lambda row: augment_data(row['Sentence']) if row['label'] in minor_classes else '', axis=1)

#df['label'].value_counts()
print(df[['augmented_sentence','label']])
selected_rows = df[df['label'].isin([1, 2, 3, 4])]
df=df[['Sentence','label']]
selected_rows=selected_rows[['augmented_sentence','label']]
selected_rows.rename(columns={'augmented_sentence':'Sentence'},inplace=True)
#df=df.drop(['augmented_sentence'],axis=1)
#df1=pd.concat([df, selected_rows])

#df1.to_csv(r"C:\Users\npvg\Documents\Multiclass\Multiclass.csv")

    augmented_sentence               label
0                          Benefit Package
1                          Benefit Package
2                          Benefit Package
3                          Benefit Package
4                          Benefit Package
..                 ...                 ...
889                              Marketing
890                        Quality of care
891                              Marketing
892                                 Access
893                     Appearance of Site

[894 rows x 2 columns]


In [14]:
# Read preprocessed CSV into TabularDataset and split it into train, test and valid.
train_data, valid_data, test_data = TabularDataset(path=f"{data_path}/Multiclass.csv", 
                                                   format='csv', 
                                                   fields=fields, 
                                                   skip_header=False).split(split_ratio=[0.70, 0.2, 0.1], 
                                                                            stratified=True, 
                                                                            strata_field='label')

In [17]:
# # Define a custom insertion function
# def insert_word(tokens, word):
#     idx = random.randint(0, len(tokens))  # Choose a random index to insert the word
#     tokens.insert(idx, word)  # Insert the word at the chosen index
#     return tokens  # Return the augmented list of tokens

# Apply data augmentation to the training data
# for example in train_data.examples:
#     tokens = example.Sentence  # Retrieve the list of tokens
#     augmented_tokens = insert_word(tokens, 'augmentation')  # Insert a word
#     example.Sentence = augmented_tokens  #
#     print(example.Sentence)

In [20]:
# Create train and validation iterators.
train_iter, valid_iter = BucketIterator.splits((train_data, valid_data),
                                               batch_size=BATCH_SIZE,
                                               device=device,
                                               shuffle=True,
                                               sort_key=lambda x: len(x.Sentence), 
                                               sort=True, 
                                               sort_within_batch=False)

In [21]:
# Test iterator, no shuffling or sorting required.
test_iter = Iterator(test_data, batch_size=BATCH_SIZE, device=device, train=False, shuffle=False, sort=False)

In [22]:
# Functions for saving and loading model parameters and metrics.
def save_checkpoint(path, model, valid_loss):
    torch.save({'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}, path)
        
def load_checkpoint(path, model):    
    state_dict = torch.load(path, map_location=device)
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

def save_metrics(path, train_loss_list, valid_loss_list, global_steps_list):   
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    torch.save(state_dict, path)

def load_metrics(path):    
    state_dict = torch.load(path, map_location=device)
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']


In [23]:
def pretrain(model, 
             optimizer, 
             train_iter, 
             valid_iter, 
             scheduler = None,
             valid_period = len(train_iter),
             num_epochs = 5):
    
    # Pretrain linear layers, do not train bert
    for param in model.roberta.parameters():
        param.requires_grad = False
    
    model.train()
    
    # Initialize losses and loss histories
    train_loss = 0.0
    valid_loss = 0.0   
    global_step = 0  
    
    # Train loop
    for epoch in range(num_epochs):
        for (source, target), _ in train_iter:
            mask = (source != PAD_INDEX).type(torch.uint8)
            
            y_pred = model(input_ids=source,  
                           attention_mask=mask)
            
            loss = torch.nn.CrossEntropyLoss()(y_pred, target)
   
            loss.backward()
            
            # Optimizer and scheduler step
            optimizer.step()    
            scheduler.step()
                
            optimizer.zero_grad()
            
            # Update train loss and global step
            train_loss += loss.item()
            global_step += 1

            # Validation loop. Save progress and evaluate model performance.
            if global_step % valid_period == 0:
                model.eval()
                
                with torch.no_grad():                    
                    for (source, target), _ in valid_iter:
                        mask = (source != PAD_INDEX).type(torch.uint8)
                        
                        y_pred = model(input_ids=source, 
                                       attention_mask=mask)
                        
                        loss = torch.nn.CrossEntropyLoss()(y_pred, target)
                        
                        valid_loss += loss.item()

                # Store train and validation loss history
                train_loss = train_loss / valid_period
                valid_loss = valid_loss / len(valid_iter)
                
                model.train()

                # print summary
                print('Epoch [{}/{}], global step [{}/{}], PT Loss: {:.4f}, Val Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_iter),
                              train_loss, valid_loss))
                
                train_loss = 0.0                
                valid_loss = 0.0
    
    # Set bert parameters back to trainable
    for param in model.roberta.parameters():
        param.requires_grad = True
        
    print('Pre-training done!')

In [24]:
def pretrain(model, 
             optimizer, 
             train_iter, 
             valid_iter, 
             scheduler = None,
             valid_period = len(train_iter),
             num_epochs = 5):
    
    # Pretrain linear layers, do not train bert
    for param in model.roberta.parameters():
        param.requires_grad = False
    
    model.train()
    
    # Initialize losses and loss histories
    train_loss = 0.0
    valid_loss = 0.0   
    global_step = 0  
    
    # Train loop
    for epoch in range(num_epochs):
        for (source, target), _ in train_iter:
            mask = (source != PAD_INDEX).type(torch.uint8)
            
            y_pred = model(input_ids=source,  
                           attention_mask=mask)
            
            loss = torch.nn.CrossEntropyLoss()(y_pred, target)
   
            loss.backward()
            
            # Optimizer and scheduler step
            optimizer.step()    
            scheduler.step()
                
            optimizer.zero_grad()
            
            # Update train loss and global step
            train_loss += loss.item()
            global_step += 1

            # Validation loop. Save progress and evaluate model performance.
            if global_step % valid_period == 0:
                model.eval()
                
                with torch.no_grad():                    
                    for (source, target), _ in valid_iter:
                        mask = (source != PAD_INDEX).type(torch.uint8)
                        
                        y_pred = model(input_ids=source, 
                                       attention_mask=mask)
                        
                        loss = torch.nn.CrossEntropyLoss()(y_pred, target)
                        
                        valid_loss += loss.item()

                # Store train and validation loss history
                train_loss = train_loss / valid_period
                valid_loss = valid_loss / len(valid_iter)
                
                model.train()

                # print summary
                print('Epoch [{}/{}], global step [{}/{}], PT Loss: {:.4f}, Val Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_iter),
                              train_loss, valid_loss))
                
                train_loss = 0.0                
                valid_loss = 0.0
    
    # Set bert parameters back to trainable
    for param in model.roberta.parameters():
        param.requires_grad = True
        
    print('Pre-training done!')

In [25]:
# Main training loop
NUM_EPOCHS = 6
steps_per_epoch = len(train_iter)

model = ROBERTAClassifier(0.4)
model = model.to(device)


optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=steps_per_epoch*1, 
                                            num_training_steps=steps_per_epoch*NUM_EPOCHS)

print("======================= Start pretraining ==============================")

pretrain(model=model,
         train_iter=train_iter,
         valid_iter=valid_iter,
         optimizer=optimizer,
         scheduler=scheduler,
         num_epochs=NUM_EPOCHS)

NUM_EPOCHS = 75
print("======================= Start training =================================")
optimizer = AdamW(model.parameters(), lr=2e-6)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=steps_per_epoch*2, 
                                            num_training_steps=steps_per_epoch*NUM_EPOCHS)

train(model=model, 
      train_iter=train_iter, 
      valid_iter=valid_iter, 
      optimizer=optimizer, 
      scheduler=scheduler, 
      num_epochs=NUM_EPOCHS)



plt.figure(figsize=(10, 8))
train_loss_list, valid_loss_list, global_steps_list = load_metrics(output_path + '/metric.pkl')
plt.plot(global_steps_list, train_loss_list, label='Train')
plt.plot(global_steps_list, valid_loss_list, label='Valid')
plt.xlabel('Global Steps', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend(fontsize=14)
plt.show()

Some weights of RobertaModel were not initialized from the model checkpoint at \\rsc.humad.com\rdrive\FinRec\FRSolutions\FRS Reporting\RXS Information\Financial Reporting\Forecast Research Dashboard\Data Science\Member_grievance\Abhirup\Model\roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


======================= Start pretraining ==============================


ValueError: invalid literal for int() with base 10: 'Benefit Package'

In [ ]:
# Evaluation Function
def evaluate(model, test_loader):
    y_pred = []
    y_true = []

    model.eval()
    with torch.no_grad():
        for (source, target), _ in test_loader:
                mask = (source != PAD_INDEX).type(torch.uint8)
                
                output = model(source, attention_mask=mask)

                y_pred.extend(torch.argmax(output, axis=-1).tolist())
                y_true.extend(target.tolist())
    
    print('Classification Report:')
    print(classification_report(y_true, y_pred, labels=[0,1,2,3,4], digits=4))
    #cm=confusion_matrix(y_true, y_pred)
    
    cm = confusion_matrix(y_true, y_pred, labels=[0,1,2,3,4])
    ax = plt.subplot()

    sns.heatmap(cm, annot=True, ax = ax, cmap='Blues', fmt="d")

    ax.set_title('Confusion Matrix')

    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('True Labels')

    ax.xaxis.set_ticklabels(['0', '1','2','3','4'])
    ax.yaxis.set_ticklabels(['0', '1','2','3','4'])



model = ROBERTAClassifier()
model = model.to(device)
load_checkpoint(output_path + '/model.pkl', model)
evaluate(model, test_iter)

In [ ]:
# Model with extra layers on top of RoBERTa
class ROBERTAClassifier(torch.nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(ROBERTAClassifier, self).__init__()
        
        self.roberta = RobertaModel.from_pretrained(r"\\rsc.humad.com\rdrive\FinRec\FRSolutions\FRS Reporting\RXS Information\Financial Reporting\Forecast Research Dashboard\Data Science\Member_grievance\Abhirup\Model\roberta-base")
        self.d1 = torch.nn.Dropout(dropout_rate)
        self.l1 = torch.nn.Linear(768, 64)
        self.bn1 = torch.nn.LayerNorm(64)
        self.d2 = torch.nn.Dropout(dropout_rate)
        self.l2 = torch.nn.Linear(64, 5)
        
    def forward(self, input_ids, attention_mask):
        _, x = self.roberta(input_ids=input_ids, attention_mask=attention_mask,return_dict=False)
        x = self.d1(x)
        x = self.l1(x)
        x = self.bn1(x)
        x = torch.nn.Tanh()(x)
        x = self.d2(x)
        x = self.l2(x)
        
        return x 

t1=time.time()
def load_checkpoint(path, model):    
    state_dict = torch.load(path, map_location=device)
    model.load_state_dict(state_dict['model_state_dict'])   
    return model,state_dict['valid_loss']

model = ROBERTAClassifier()
model = model.to(device)
saved_model_path = output_path + '/model.pkl'  # Adjust this according to your setup
loaded_model, _ = load_checkpoint(saved_model_path,model)

In [ ]:
# Load pre-trained model tokenizer
tokenizer = RobertaTokenizer.from_pretrained(r"\\rsc.humad.com\rdrive\FinRec\FRSolutions\FRS Reporting\RXS Information\Financial Reporting\Forecast Research Dashboard\Data Science\Member_grievance\Abhirup\Model\roberta-base")
t2=time.time()
t2-t1

def predict_sentences_batch(model, tokenizer, sentences, batch_size=32):
    predicted_labels = []
    dataloader = DataLoader(sentences, batch_size=batch_size)

    for batch in dataloader:
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        predicted_labels.extend(torch.argmax(outputs, dim=-1).tolist())
    return predicted_labels

t1 = time.time()
predicted_labels = predict_sentences_batch(loaded_model, tokenizer, sentences)
t2 = time.time()
print("Time taken:", t2 - t1)